#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [6]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [7]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embeddings)
db

In [8]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [9]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 (Manhattan) distance. Therefore, a lower score is better.

In [10]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='8646139a-4f44-4cc4-8f28-c1b6e6874285', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  15389.117),
 (Document(id='8495c732-a1e1-48db-970a-a18ad8af297c', metadata={'source': 'speech.txt'},

Passing query as embedded vector

In [11]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.877864122390747,
 -3.1047275066375732,
 1.940829873085022,
 1.4896199703216553,
 -0.8771411180496216,
 0.6832279562950134,
 1.517905592918396,
 -0.5397210717201233,
 0.950010359287262,
 -2.017749309539795,
 1.2945266962051392,
 -1.9506374597549438,
 -0.6536946296691895,
 1.5775196552276611,
 -0.5198744535446167,
 -1.9926263093948364,
 -0.3562990427017212,
 0.10658346116542816,
 0.712094247341156,
 -1.8072690963745117,
 -0.6980193853378296,
 -1.0365055799484253,
 2.133053779602051,
 -1.9812206029891968,
 0.7362547516822815,
 -0.991586446762085,
 0.3820662200450897,
 -1.6680575609207153,
 0.1732770949602127,
 -0.8440052270889282,
 2.1305246353149414,
 -2.8417069911956787,
 -2.6762630939483643,
 3.789302349090576,
 2.2782208919525146,
 -5.046081066131592,
 -1.0799129009246826,
 1.3606904745101929,
 -1.2617709636688232,
 -1.4509692192077637,
 -0.49043193459510803,
 -2.0093657970428467,
 2.0919744968414307,
 0.3043014407157898,
 0.9796836972236633,
 0.5794857740402222,
 -0.28861439228057

In [12]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='8646139a-4f44-4cc4-8f28-c1b6e6874285', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='8495c732-a1e1-48db-970a-a18ad8af297c', metadata={'source': 'speech.txt'}, page_content='â

In [13]:
### Saving And Loading
db.save_local("faiss_index")

In [14]:
# Loading the local db
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [15]:
docs

[Document(id='8646139a-4f44-4cc4-8f28-c1b6e6874285', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='8495c732-a1e1-48db-970a-a18ad8af297c', metadata={'source': 'speech.txt'}, page_content='â